In [2]:
%load_ext autoreload
%autoreload 2

import optuna
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import shap
import numpy as np
import scipy
import seaborn as sns

from steps.prepare_data import load_split_processed_data, process_train_data, process_test_data
from utils.model import predict_booster, load_model, Metrics, predict_xgbm, predict

In [6]:
# process_train_data()
# process_test_data()

train_data, test_data = load_split_processed_data()

model_name = "LightGbmV2_new_features_v7_tuning_v3.pickle"

print("-----------Train----------")
predict_booster(model_name, train_data)

print("-----------Test----------")
predict_booster(model_name, test_data)

None

-----------Train----------

Metrics
AUC: 0.97
Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98    140413
           1       0.87      0.48      0.62      9585

    accuracy                           0.96    149998
   macro avg       0.92      0.74      0.80    149998
weighted avg       0.96      0.96      0.96    149998

-----------Test----------

Metrics
AUC: 0.90
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97    140597
           1       0.68      0.34      0.46      9403

    accuracy                           0.95    150000
   macro avg       0.82      0.67      0.71    150000
weighted avg       0.94      0.95      0.94    150000



In [5]:
pd.set_option('display.max_columns', None)

train_data

,num_act_days_mea_wk1,device_brand_other,num_act_days_min_mnt1,active_ppm,num_act_days_min_mnt3,num_act_days_mea_mnt1,loc_market_share,voice_mts_in_dwork_part_min_mnt1,voice_in_kievstar_part_std_mnt1,device_type_smartphone,voice_mts_in_drest_part_mea_wk1,bs_count_ppm_mn3,device_sim_count,MV_dou_net,num_act_days_td_mnt3,device_days_usage,device_has_lte,num_act_days_std_mnt1,voice_mts_in_drest_part_std_mnt1,days_of_end_last_ppm,voice_mts_in_dwork_part_std_mnt1,lt,voice_mts_in_dwork_part_min_mnt3,conn_in_uniq_cnt_td_mnt3,voice_in_life_part_std_mnt3,Balance_uah,ks_num_part_mea_wk1,day_end_gba,voice_mts_in_dwork_part_mea_mnt3,com_num_part_td_mnt3,device_android_version,num_act_days_std_mnt3,voice_mts_out_drest_partmea_mnt1,loc_is_obl_center,num_act_days_max_mnt1,conn_com_part_min_mnt1,voice_mts_out_dwork_partmea_mnt3,device_type_phone,voice_mts_out_drest_partmea_wk1,voice_mts_out_dwork_partstd_mnt1,device_height_mm,is_avg_paym,content_cnt_mea_mnt3,cnt_paym_6_month,device_brand_nan,MV_DOU_OT,voice_mts_in_drest_part_td_mnt3,voice_mts_in_drest_part_mea_mnt3,tsoa_call_cnt,device_width_mm,voice_mts_out_dwork_partmin_mnt1,voice_in_life_part_td_mnt3,is_max_paym,voice_in_short_part_td_mnt3,num_act_days_mea_mnt3,device_has_wifi,voice_mts_in_dwork_part_td_mnt3,imei_mean_days_usage,voice_mts_out_nrest_partmea_wk1,voice_out_fix_tar_dur_max_mnt3,accum_oth_dur_std_mnt1,conn_com_part_td_mnt3,voice_mts_out_dwork_partmin_mnt3,ks_num_part_std_mnt1,MV_Traf_4G_d_Mb,all_cnt_std_mnt1,myvf_day_usage,accum_mts_dur_max_mnt3,data_3g_dou_td_mnt3,sum_paym_6_month,sms_in_cnt_max_mnt3,all_cnt_std_mnt3,voice_mts_out_nwork_partmin_mnt1,MV_dou_4G_d,gprs_tar_vol_min_mnt1,ks_num_part_std_mnt3,abon_id,MV_DOU_AP,voice_mts_in_nwork_part_td_mnt3,data_3g_tv_cnt_std_mnt3,sms_in_cnt_mea_wk1,voice_mts_out_drest_partmin_mnt1,sms_in_cnt_mea_mnt3,days_of_last_ppm,imei_mean_long_days_usage,sms_in_cnt_std_mnt3,voice_in_tar_dur_min_mnt1,voice_in_td_cnt_td_mnt3,voice_out_tar_dur_min_mnt3,target,MAX_of_day_cnt_susp_app_sum,MAX_of_day_cnt_susp_app_mean,MAX_of_day_cnt_susp_app_share,SUM_of_Count_events_susp_app_sum,SUM_of_Count_events_susp_app_mean,SUM_of_Count_events_susp_app_share,SUM_of_Duration_sec_susp_app_sum,SUM_of_Duration_sec_susp_app_mean,SUM_of_Duration_sec_susp_app_share,SUM_of_Volume_kb_susp_app_sum,SUM_of_Volume_kb_susp_app_mean,SUM_of_Volume_kb_susp_app_share,MAX_of_day_cnt_sum,MAX_of_day_cnt_mean,SUM_of_Count_events_sum,SUM_of_Count_events_mean,SUM_of_Duration_sec_sum,SUM_of_Duration_sec_mean,SUM_of_Volume_kb_sum,SUM_of_Volume_kb_mean,MAX_of_day_cnt_top_apps_sum,MAX_of_day_cnt_top_apps_mean,SUM_of_Count_events_top_apps_sum,SUM_of_Count_events_top_apps_mean,SUM_of_Duration_sec_top_apps_sum,SUM_of_Duration_sec_top_apps_mean,SUM_of_Volume_kb_top_apps_sum,SUM_of_Volume_kb_top_apps_mean,vodafone_act_share,vodafone_act_count,casual_act_share,casual_act_count,finance_topic_topic_act_share,finance_topic_topic_act_count,vodafone_topic_topic_act_share,vodafone_topic_topic_act_count,casual_topic_topic_act_share,casual_topic_topic_act_count,competitors_topic_topic_act_share,competitors_topic_topic_act_count,messengars_topic_topic_act_share,messengars_topic_topic_act_count,credits_companies_category_act_share,credits_companies_category_act_count,messengars_companies_category_act_share,messengars_companies_category_act_count,vodafone_support_category_act_share,vodafone_support_category_act_count,vodafone_survey_category_act_share,vodafone_survey_category_act_count,vodafone_services_category_act_share,vodafone_services_category_act_count,bank_numbers_category_act_share,bank_numbers_category_act_count,phone_companies_category_act_share,phone_companies_category_act_count,casual_topic_call_cnt_out_sum,competitors_topic_call_cnt_out_sum,finance_topic_call_cnt_out_sum,messengars_topic_call_cnt_out_sum,vodafone_topic_call_cnt_out_sum,casual_topic_call_cnt_out_sum_share,competitors_topic_call_cnt_out_sum_share,finance_topic_call_cnt_out_sum_share,messengars_topic_call_cnt_out_sum_share,vod